In [1]:
import subprocess
import os

result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value

In [2]:
#模型下载
from modelscope import snapshot_download
model_dir = snapshot_download('unsloth/Qwen2.5-VL-7B-Instruct',local_dir='/root/autodl-tmp/qwvl')

2025-05-23 21:47:32,715 - modelscope - INFO - Got 18 files, start to download ...


Processing 18 items:   0%|          | 0.00/18.0 [00:00<?, ?it/s]

2025-05-23 22:09:20,829 - modelscope - INFO - Download model 'unsloth/Qwen2.5-VL-7B-Instruct' successfully.
2025-05-23 22:09:20,831 - modelscope - INFO - Target directory already exists, skipping creation.


In [3]:
from unsloth import FastVisionModel # FastLanguageModel for LLMs
from IPython.display import display, Math, Latex
import torch

model, tokenizer = FastVisionModel.from_pretrained(
    model_dir,
    load_in_4bit = True, # Use 4bit to reduce memory use. False for 16bit LoRA.
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for long context
)

print(model)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
[2025-05-23 22:09:47,657] [INFO] [real_accelerator.py:239:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/root/miniconda3/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/root/miniconda3/compiler_compat/ld: warning: libpthread.so.0, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/root/miniconda3/compiler_compat/ld: warning: libstdc++.so.6, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/root/miniconda3/compiler_compat/ld: warning: libm.so.6, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/root/miniconda3/compiler_compat/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `std::runtime_error::~runtime_error()@GLIBCXX_3.4'
/root/miniconda3/compiler_compat/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `__gxx_personality_v0@CXXABI_1.3'
/root/miniconda3/compiler_compat/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `std::ostream::tellp()@GLIBCXX_3.4'
/root/minicond

🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.5.6: Fast Qwen2 patching. Transformers: 4.51.3.
   \\   /|    NVIDIA GeForce RTX 3090. Num GPUs = 1. Max memory: 23.57 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 8.6. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


Qwen2_5_VLForConditionalGeneration(
  (visual): Qwen2_5_VisionTransformerPretrainedModel(
    (patch_embed): Qwen2_5_VisionPatchEmbed(
      (proj): Conv3d(3, 1280, kernel_size=(2, 14, 14), stride=(2, 14, 14), bias=False)
    )
    (rotary_pos_emb): Qwen2_5_VisionRotaryEmbedding()
    (blocks): ModuleList(
      (0-31): 32 x Qwen2_5_VLVisionBlock(
        (norm1): Qwen2RMSNorm((1280,), eps=1e-06)
        (norm2): Qwen2RMSNorm((1280,), eps=1e-06)
        (attn): Qwen2_5_VLVisionSdpaAttention(
          (qkv): Linear4bit(in_features=1280, out_features=3840, bias=True)
          (proj): Linear4bit(in_features=1280, out_features=1280, bias=True)
        )
        (mlp): Qwen2_5_VLMLP(
          (gate_proj): Linear4bit(in_features=1280, out_features=3420, bias=True)
          (up_proj): Linear4bit(in_features=1280, out_features=3420, bias=True)
          (down_proj): Linear4bit(in_features=3420, out_features=1280, bias=True)
          (act_fn): SiLU()
        )
      )
    )
    (merger): Q

In [4]:
model = FastVisionModel.get_peft_model(
    model,
    finetune_vision_layers     = True, # False if not finetuning vision layers
    finetune_language_layers   = True, # False if not finetuning language layers
    finetune_attention_modules = True, # False if not finetuning attention layers
    finetune_mlp_modules       = True, # False if not finetuning MLP layers

    r = 8,           # The larger, the higher the accuracy, but might overfit
    lora_alpha = 16,  # Recommended alpha == r at least
    lora_dropout = 0.1,
    bias = "none",
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
    # target_modules = "all-linear", # Optional now! Can specify a list if needed
)

In [6]:
from datasets import load_dataset
dataset = load_dataset("unsloth/LaTeX_OCR", split = "train")

test-00000-of-00001.parquet:   0%|          | 0.00/38.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/68686 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7632 [00:00<?, ? examples/s]

In [7]:
#查看数据元数据信息
dataset

Dataset({
    features: ['image', 'text'],
    num_rows: 68686
})

In [8]:
latex = dataset[2]["text"]
display(Math(latex))

<IPython.core.display.Math object>

In [9]:
dataset[0]

{'image': <PIL.PngImagePlugin.PngImageFile image mode=RGB size=160x40>,
 'text': '{ \\frac { N } { M } } \\in { \\bf Z } , { \\frac { M } { P } } \\in { \\bf Z } , { \\frac { P } { Q } } \\in { \\bf Z }'}

#### 所有的多模态模型 微调，数据格式都为
```
[
{ "role": "user",
  "content": [{"type": "text",  "text": Q}, {"type": "image", "image": image} ]
},
{ "role": "assistant",
  "content": [{"type": "text",  "text": A} ]
},
]
```

In [10]:

#调整数据集
instruction = "Write the LaTeX representation for this image."

def convert_to_conversation(sample):
    conversation = [
        { "role": "user",
          "content" : [
            {"type" : "text",  "text"  : instruction},
            {"type" : "image", "image" : sample["image"]} ]
        },
        { "role" : "assistant",
          "content" : [
            {"type" : "text",  "text"  : sample["text"]} ]
        },
    ]
    return { "messages" : conversation }

In [11]:
converted_dataset = [convert_to_conversation(sample) for sample in dataset]

In [40]:
converted_dataset[2]

{'messages': [{'role': 'user',
   'content': [{'type': 'text',
     'text': 'Write the LaTeX representation for this image.'},
    {'type': 'image',
     'image': <PIL.PngImagePlugin.PngImageFile image mode=RGB size=320x50>}]},
  {'role': 'assistant',
   'content': [{'type': 'text',
     'text': 'H ^ { \\prime } = \\beta N \\int d \\lambda \\biggl \\{ \\frac { 1 } { 2 \\beta ^ { 2 } N ^ { 2 } } \\partial _ { \\lambda } \\zeta ^ { \\dagger } \\partial _ { \\lambda } \\zeta + V ( \\lambda ) \\zeta ^ { \\dagger } \\zeta \\biggr \\} \\ .'}]}]}

In [13]:
#模型 微调前 推理测试
FastVisionModel.for_inference(model) # Enable for inference!

image = dataset[2]["image"]
instruction = "Write the LaTeX representation for this image."

messages = [
    {"role": "user", "content": [
        {"type": "image"},
        {"type": "text", "text": instruction}
    ]}
]
input_text = tokenizer.apply_chat_template(messages, add_generation_prompt = True)
inputs = tokenizer(
    image,
    input_text,
    add_special_tokens = False,
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128,
                   use_cache = True, temperature = 1.5, min_p = 0.1)

```latex
H' = \beta N \int d\lambda \left\{\frac{1}{2\beta^2N^{2}}\partial_\lambda\zeta^*\partial_\lambda\zeta + V(\lambda)\zeta^*\zeta\right\}.
```<|im_end|>


#### 模型trainer

In [14]:
from unsloth import is_bf16_supported
from unsloth.trainer import UnslothVisionDataCollator
from trl import SFTTrainer, SFTConfig

FastVisionModel.for_training(model) # Enable for training!

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    data_collator = UnslothVisionDataCollator(model, tokenizer), # Must use!
    train_dataset = converted_dataset,
    args = SFTConfig(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 30,
        # num_train_epochs = 1, # Set this instead of max_steps for full training runs
        learning_rate = 2e-4,
        fp16 = not is_bf16_supported(),
        bf16 = is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "swanlab",     # For Weights and Biases

        # You MUST put the below items for vision finetuning:
        remove_unused_columns = False,
        dataset_text_field = "",
        dataset_kwargs = {"skip_prepare_dataset": True},
        dataset_num_proc = 4,
        max_seq_length = 2048,
    ),
)

Unsloth: Model does not have a default image size - using 512


In [15]:
#Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA GeForce RTX 3090. Max memory = 23.57 GB.
7.854 GB of memory reserved.


In [16]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 68,686 | Num Epochs = 1 | Total steps = 30
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 25,760,768/4,740,300,800 (0.54% trained)


swanlab: Tracking run with swanlab version 0.5.8                                   
swanlab: Run data will be saved locally in /root/swanlog/run-20250523_221444-45da561d
swanlab: 👋 Hi upstone, welcome to swanlab!
swanlab: Syncing run outputs to the cloud
swanlab: 🏠 View project at https://swanlab.cn/@upstone/root
swanlab: 🚀 View run at https://swanlab.cn/@upstone/root/runs/sd8vkpnh3hq2qlsto5t33


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,2.766900
2,3.286300
3,3.472100
4,2.546300
5,2.352200
6,2.159200
7,1.592000
8,1.056600
9,0.762000
10,0.787300


swanlab: Step 30 on key train/epoch already exists, ignored.
swanlab: Step 30 on key train/global_step already exists, ignored.


In [17]:
#Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

92.3381 seconds used for training.
1.54 minutes used for training.
Peak reserved memory = 7.854 GB.
Peak reserved memory for training = 0.0 GB.
Peak reserved memory % of max memory = 33.322 %.
Peak reserved memory for training % of max memory = 0.0 %.


In [25]:
FastVisionModel.for_inference(model) # Enable for inference!

image = dataset[2]["image"]
instruction = "Write the LaTeX representation for this image."

messages = [
    {"role": "user", "content": [
        {"type": "image"},
        {"type": "text", "text": instruction}
    ]}
]
input_text = tokenizer.apply_chat_template(messages, add_generation_prompt = True)
inputs = tokenizer(
    image,
    input_text,
    add_special_tokens = False,
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
output_content = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128,
                   use_cache = True, temperature = 1.5, min_p = 0.1)

H ^ { \prime } = \beta N \int d \lambda \Biggl \{ \frac { 1 } { 2 \beta ^ { 3 / 2 } N ^ { 2 } } \partial _ { t } \zeta ^ { t } \partial _ { x } \zeta + V ( \lambda ) \zeta ^ { t } \zeta \Biggr \} ~ .<|im_end|>


In [29]:
output_content

tensor([[151644,   8948,    198,   2610,    525,    264,  10950,  17847,     13,
         151645,    198, 151644,    872,    198, 151652, 151655, 151655, 151655,
         151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655,
         151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655,
         151655, 151653,   7985,    279,  97913,  13042,    369,    419,   2168,
             13, 151645,    198, 151644,  77091,    198,     39,   6306,    314,
           1124,  32338,    335,    284,   1124,  19127,    451,   1124,    396,
            294,   1124,  12935,   1124,  15636,   6072,   1124,     90,   1124,
          37018,    314,    220,     16,    335,    314,    220,     17,   1124,
          19127,   6306,    314,    220,     18,    608,    220,     17,    335,
            451,   6306,    314,    220,     17,    335,    335,   1124,  37420,
            716,    314,    259,    335,   1124,     89,   1915,   6306,    314,
            259,    335,   1

In [32]:
tokenizer.decode(output_content[0],skip_prompt = True).split("assistant\n")[1]

'H ^ { \\prime } = \\beta N \\int d \\lambda \\Biggl \\{ \\frac { 1 } { 2 \\beta ^ { 3 / 2 } N ^ { 2 } } \\partial _ { t } \\zeta ^ { t } \\partial _ { x } \\zeta + V ( \\lambda ) \\zeta ^ { t } \\zeta \\Biggr \\} ~ .<|im_end|>'

In [19]:
model.save_pretrained("/root/autodl-tmp/lora_model")  # Local saving
tokenizer.save_pretrained("/root/autodl-tmp/lora_model")

[]

In [23]:
model.save_pretrained_merged("/root/autodl-tmp/unsloth_finetune", tokenizer,)

Detected local model directory: /root/autodl-tmp/qwvl
Found HuggingFace hub cache directory: /root/.cache/huggingface/hub
Copying safetensors from local directory: /root/autodl-tmp/qwvl
Copied safetensors index file from local model


Unsloth: Merging weights into 16bit:   0%|          | 0/4 [00:00<?, ?it/s]

Copied model-00004-of-00004.safetensors from local model directory


Unsloth: Merging weights into 16bit:  25%|██▌       | 1/4 [00:03<00:11,  3.68s/it]

Copied model-00001-of-00004.safetensors from local model directory


Unsloth: Merging weights into 16bit:  50%|█████     | 2/4 [00:13<00:15,  7.52s/it]

Copied model-00002-of-00004.safetensors from local model directory


Unsloth: Merging weights into 16bit:  75%|███████▌  | 3/4 [00:24<00:09,  9.13s/it]

Copied model-00003-of-00004.safetensors from local model directory


Unsloth: Merging weights into 16bit: 100%|██████████| 4/4 [00:35<00:00,  8.80s/it]


In [43]:
model.save_pretrained_gguf("/root/autodl-tmp/gg", tokenizer)

Unsloth: Updating system package directories
Unsloth: Install GGUF and other packages


AssertionError: 

In [35]:
help(model.save_pretrained_gguf)

Help on method save_to_gguf_generic in module unsloth.save:

save_to_gguf_generic(save_directory, quantization_type='Q8_0', repo_id=None, token=None) method of peft.peft_model.PeftModelForCausalLM instance

